<a href="https://colab.research.google.com/github/Gaurav-Ramachandra/AI-Lab/blob/main/AI_Lab8_26_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class KnowledgeBase:
    def __init__(self):
        self.facts = set()
        self.rules = []

    def add_fact(self, fact):
        self.facts.add(fact)

    def add_rule(self, rule):
        self.rules.append(rule)

    def __repr__(self):
        return f"Facts: {self.facts}\nRules: {self.rules}"


def parse_fact_or_rule(input_str):
    """
    Parses a user input string into a fact or rule.
    Facts are strings like 'A'.
    Rules are strings like 'A,B => C' (comma-separated premises imply a conclusion).
    """
    if "=>" in input_str:
        premises, conclusion = input_str.split("=>")
        premises = [premise.strip() for premise in premises.split(",")]
        return premises, conclusion.strip()
    else:
        return input_str.strip()


def forward_reasoning(kb, query):
    """
    Perform forward reasoning on the knowledge base to prove the query.
    """
    inferred = set()
    while True:
        new_facts = set()
        for rule in kb.rules:
            premises, conclusion = rule
            if all(premise in kb.facts for premise in premises):
                if conclusion not in kb.facts:
                    new_facts.add(conclusion)

        if not new_facts:
            break

        kb.facts.update(new_facts)
        inferred.update(new_facts)

        if query in kb.facts:
            return True, inferred

    return False, inferred


def interactive_prover():
    kb = KnowledgeBase()
    print("Forward Reasoning Interactive Prover")
    print("Enter facts and rules. Use 'A' for a fact or 'A,B => C' for a rule.")
    print("Type 'done' when finished adding to the knowledge base.")

    while True:
        user_input = input("Enter a fact or rule: ")
        if user_input.lower() == "done":
            break

        if "=>" in user_input:
            rule = parse_fact_or_rule(user_input)
            kb.add_rule(rule)
        else:
            fact = parse_fact_or_rule(user_input)
            kb.add_fact(fact)

    print("\nKnowledge Base Constructed:")
    print(kb)

    query = input("\nEnter a query to prove: ").strip()
    result, inferred = forward_reasoning(kb, query)

    if result:
        print(f"The query '{query}' was proven using forward reasoning.")
        print(f"Inferred facts: {inferred}")
    else:
        print(f"The query '{query}' could not be proven.")
        print(f"Inferred facts: {inferred}")


# Run the interactive prover
interactive_prover()


Forward Reasoning Interactive Prover
Enter facts and rules. Use 'A' for a fact or 'A,B => C' for a rule.
Type 'done' when finished adding to the knowledge base.
Enter a fact or rule: a
Enter a fact or rule: b
Enter a fact or rule: a,b=>c
Enter a fact or rule: c=>d
Enter a fact or rule: done

Knowledge Base Constructed:
Facts: {'b', 'a'}
Rules: [(['a', 'b'], 'c'), (['c'], 'd')]

Enter a query to prove: d
The query 'd' was proven using forward reasoning.
Inferred facts: {'c', 'd'}


In [ ]:
class FirstOrderLogicKB:
    def __init__(self):
        """Initialize the knowledge base with an empty set of facts and rules."""
        self.facts = set()  # Set to hold facts
        self.rules = []     # List to hold rules (premise, conclusion)

    def add_fact(self, fact):
        """Add a fact to the knowledge base."""
        self.facts.add(fact)

    def add_rule(self, premise, conclusion):
        """Add a rule to the knowledge base: premise => conclusion."""
        self.rules.append((premise, conclusion))

    def forward_reasoning(self):
        """Apply forward reasoning to deduce new facts from the knowledge base."""
        new_facts = set(self.facts)
        inferred = True

        # Keep inferring new facts until no new facts can be deduced
        while inferred:
            inferred = False
            for premise, conclusion in self.rules:
                if premise.issubset(new_facts) and conclusion not in new_facts:
                    new_facts.add(conclusion)
                    inferred = True
        self.facts = new_facts

    def query(self, fact):
        """Check if a fact is in the knowledge base after reasoning."""
        return fact in self.facts

    def display_facts(self):
        """Display all the facts in the knowledge base."""
        print("Known Facts:")
        for fact in self.facts:
            print(fact)


def input_facts_and_rules():
    kb = FirstOrderLogicKB()

    # Input facts
    kb.add_fact("American(Robert)")
    kb.add_fact("Enemy(A, America)")
    kb.add_fact("Owns(A, T1)")
    kb.add_fact("Missile(T1)")
    kb.add_fact("SellsWeapon(Robert, T1)")

    # Input rule for being a criminal
    rule_premise = {"American(Robert)", "SellsWeapon(Robert, T1)", "Enemy(A, America)"}
    rule_conclusion = "Criminal(Robert)"
    kb.add_rule(rule_premise, rule_conclusion)

    # Perform forward reasoning to deduce new facts
    kb.forward_reasoning()

    return kb

def main():
    # Input the knowledge base facts and rules
    kb = input_facts_and_rules()

    # Display all the facts entered
    kb.display_facts()

    # Query the knowledge base to prove if Robert is a criminal
    if kb.query("Criminal(Robert)"):
        print("\nYes, Robert is a criminal.")
    else:
        print("\nNo, Robert is not a criminal.")

if __name__ == "__main__":
    main()

Known Facts:
Criminal(Robert)
Owns(A, T1)
American(Robert)
SellsWeapon(Robert, T1)
Missile(T1)
Enemy(A, America)

Yes, Robert is a criminal.


In [ ]:
class ForwardChainingFOL:
    def __init__(self):
        self.facts = set()  # Set of known facts
        self.rules = []  # List of rules in the form (premises, conclusion)

    def add_fact(self, fact):
        self.facts.add(fact)

    def add_rule(self, premises, conclusion):
        self.rules.append((premises, conclusion))

    def unify(self, fact1, fact2):
        """
        Unifies two facts if possible. Returns a substitution dictionary or None if unification fails.
        """
        if fact1 == fact2:
            return {}  # No substitution needed
        if "(" in fact1 and "(" in fact2:
            # Split into predicate and arguments
            pred1, args1 = fact1.split("(", 1)
            pred2, args2 = fact2.split("(", 1)
            args1 = args1[:-1].split(",")
            args2 = args2[:-1].split(",")
            if pred1 != pred2 or len(args1) != len(args2):
                return None
            # Unify arguments
            substitution = {}
            for a1, a2 in zip(args1, args2):
                if a1 != a2:
                    if a1.islower():  # a1 is a variable
                        substitution[a1] = a2
                    elif a2.islower():  # a2 is a variable
                        substitution[a2] = a1
                    else:  # Both are constants and different
                        return None
            return substitution
        return None

    def apply_substitution(self, fact, substitution):
        """
        Applies a substitution to a fact and returns the substituted fact.
        """
        if "(" in fact:
            pred, args = fact.split("(", 1)
            args = args[:-1].split(",")
            substituted_args = [substitution.get(arg, arg) for arg in args]
            return f"{pred}({','.join(substituted_args)})"
        return fact

    def forward_chain(self, goal):
        iteration = 1
        while True:
            new_facts = set()
            print(f"\n=== Iteration {iteration} ===")
            print("Known Facts:")
            for fact in self.facts:
                print(f"  - {fact}")

            print("\nApplying rules...")
            rule_triggered = False

            for premises, conclusion in self.rules:
                substitutions = [{}]
                for premise in premises:
                    new_substitutions = []
                    for fact in self.facts:
                        for sub in substitutions:
                            unified = self.unify(self.apply_substitution(premise, sub), fact)
                            if unified is not None:
                                new_substitutions.append({**sub, **unified})
                    substitutions = new_substitutions
                for sub in substitutions:
                    inferred_fact = self.apply_substitution(conclusion, sub)
                    if inferred_fact not in self.facts:
                        rule_triggered = True
                        print(f"Rule triggered: {premises} → {conclusion}")
                        print(f"  New fact inferred: {inferred_fact}")
                        new_facts.add(inferred_fact)

            if not new_facts:
                if not rule_triggered:
                    print("No rules triggered in this iteration.")
                print("No new facts inferred in this iteration.")
                break

            self.facts.update(new_facts)
            if goal in self.facts:
                print(f"\nGoal {goal} reached!")
                return True
            iteration += 1

        print("\nGoal not reached.")
        return False


# Problem setup
fc = ForwardChainingFOL()

# Facts
fc.add_fact("American(Robert)")
fc.add_fact("Enemy(A,America)")
fc.add_fact("Owns(A,T1)")
fc.add_fact("Missile(T1)")

# Rules
fc.add_rule(["Missile(T1)"], "Weapon(T1)")
fc.add_rule(["Enemy(A,America)"], "Hostile(A)")
fc.add_rule(["Missile(p)", "Owns(A,p)"], "Sells(Robert,p,A)")
fc.add_rule(["American(p)", "Weapon(q)", "Sells(p,q,r)", "Hostile(r)"], "Criminal(p)")

# Goal
goal = "Criminal(Robert)"

# Perform forward chaining
if fc.forward_chain(goal):
    print(f"\nFinal result: Goal achieved: {goal}")
else:
    print("\nFinal result: Goal not achieved.")



=== Iteration 1 ===
Known Facts:
  - Missile(T1)
  - American(Robert)
  - Owns(A,T1)
  - Enemy(A,America)

Applying rules...
Rule triggered: ['Missile(T1)'] → Weapon(T1)
  New fact inferred: Weapon(T1)
Rule triggered: ['Enemy(A,America)'] → Hostile(A)
  New fact inferred: Hostile(A)
Rule triggered: ['Missile(p)', 'Owns(A,p)'] → Sells(Robert,p,A)
  New fact inferred: Sells(Robert,T1,A)

=== Iteration 2 ===
Known Facts:
  - American(Robert)
  - Enemy(A,America)
  - Owns(A,T1)
  - Missile(T1)
  - Hostile(A)
  - Sells(Robert,T1,A)
  - Weapon(T1)

Applying rules...
Rule triggered: ['American(p)', 'Weapon(q)', 'Sells(p,q,r)', 'Hostile(r)'] → Criminal(p)
  New fact inferred: Criminal(Robert)

Goal Criminal(Robert) reached!

Final result: Goal achieved: Criminal(Robert)
